# Assignement 1

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report
import skfuzzy as fuzz
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import seaborn as sns
from sklearn import datasets

In [ ]:
#Load Dataset 
diabetes = datasets.load_diabetes(as_frame = True)
X = diabetes.data.values
y = diabetes.target.values

diabetes.frame.head()

In [ ]:
#train test spliting
test_size=0.2
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
# Standardize features
scaler=StandardScaler()
Xtr= scaler.fit_transform(Xtr)
Xte= scaler.transform(Xte)

In [ ]:
# Number of clusters 
n_clusters = 2
m=2

# Concatenate target for clustering
Xexp= np.concatenate([Xtr, ytr.reshape(-1,1)], axis=1)
#Xexp=Xtr

# Transpose data for skfuzzy (expects features x samples)
Xexp_T = Xexp.T 

# Fuzzy C-means clustering
centers, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    Xexp_T, n_clusters, m=m, error=0.005, maxiter=1000, init=None,
)

In [ ]:
# Compute sigma (spread) for each cluster
sigmas = []
for j in range(n_clusters):
    # membership weights for cluster j, raised to m
    u_j = u[j, :] ** m
    # weighted variance for each feature
    var_j = np.average((Xexp - centers[j])**2, axis=0, weights=u_j)
    sigma_j = np.sqrt(var_j)
    sigmas.append(sigma_j)
sigmas=np.array(sigmas)# Hard clustering from fuzzy membership

# Hard clustering from fuzzy membership
cluster_labels = np.argmax(u, axis=0)
print("Fuzzy partition coefficient (FPC):", fpc)

In [ ]:
cluster_labels = np.argmax(u, axis=0)  

# Turn Xexp into a dataframe for plotting
df = pd.DataFrame(Xexp, columns=[f"Feature {i}" for i in range(Xexp.shape[1])])
df["Cluster"] = cluster_labels

# Pairplot with hue = cluster
sns.pairplot(df, vars=df.columns[:-1], hue="Cluster", palette="tab10", diag_kind="kde")
plt.show()

In [ ]:
# ---------------------------
# Gaussian Membership Function
# ---------------------------
class GaussianMF(nn.Module):
    def __init__(self, centers, sigmas, agg_prob):
        super().__init__()
        self.centers = nn.Parameter(torch.tensor(centers, dtype=torch.float32))
        self.sigmas = nn.Parameter(torch.tensor(sigmas, dtype=torch.float32))
        self.agg_prob=agg_prob

    def forward(self, x):
        # Expand for broadcasting
        # x: (batch, 1, n_dims), centers: (1, n_rules, n_dims), sigmas: (1, n_rules, n_dims)
        diff = abs((x.unsqueeze(1) - self.centers.unsqueeze(0))/self.sigmas.unsqueeze(0)) #(batch, n_rules, n_dims)

        # Aggregation
        if self.agg_prob:
            dist = torch.norm(diff, dim=-1)  # (batch, n_rules) # probablistic intersection
        else:
            dist = torch.max(diff, dim=-1).values  # (batch, n_rules) # min intersection (min instersection of normal funtion is the same as the max on dist)
        
        return torch.exp(-0.5 * dist ** 2)


# ---------------------------
# TSK Model
# ---------------------------
class TSK(nn.Module):
    def __init__(self, n_inputs, n_rules, centers, sigmas,agg_prob=False):
        super().__init__()
        self.n_inputs = n_inputs
        self.n_rules = n_rules

        # Antecedents (Gaussian MFs)
        
        self.mfs=GaussianMF(centers, sigmas,agg_prob) 

        # Consequents (linear functions of inputs)
        # Each rule has coeffs for each input + bias
        self.consequents = nn.Parameter(
            torch.randn(n_inputs + 1,n_rules)
        )

    def forward(self, x):
        # x: (batch, n_inputs)
        batch_size = x.shape[0]
        
        # Compute membership values for each input feature
        # firing_strengths: (batch, n_rules)
        firing_strengths = self.mfs(x)
        
        # Normalize memberships
        # norm_fs: (batch, n_rules)
        norm_fs = firing_strengths / (firing_strengths.sum(dim=1, keepdim=True) + 1e-9)

        # Consequent output (linear model per rule)
        x_aug = torch.cat([x, torch.ones(batch_size, 1)], dim=1)  # add bias

        rule_outputs = torch.einsum("br,rk->bk", x_aug, self.consequents)  # (batch, rules)
        # Weighted sum
        output = torch.sum(norm_fs * rule_outputs, dim=1, keepdim=True)

        return output, norm_fs, rule_outputs

In [ ]:
# ---------------------------
# Gradient Descent Training 
# ---------------------------
def train_gd(model, X, y, epochs=100, lr=1e-3):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    for _ in range(epochs):
        optimizer.zero_grad()
        y_pred, _, _ = model(X)
        loss = criterion(y_pred, y)
        #print(loss)
        loss.backward()
        optimizer.step()

In [ ]:
# ---------------------------
# Hybrid Training (Classic ANFIS)
# ---------------------------
def train_hybrid_anfis(model, X, y, max_iters=10, gd_epochs=20, lr=1e-3):
    train_ls(model, X, y)
    for _ in range(max_iters):
        # Step A: GD on antecedents (freeze consequents)
        model.consequents.requires_grad = False
        train_gd(model, X, y, epochs=gd_epochs, lr=lr)

        # Step B: LS on consequents (freeze antecedents)
        model.consequents.requires_grad = True
        model.mfs.requires_grad = False
        train_ls(model, X, y)

        # Re-enable antecedents
        model.mfs.requires_grad = True

In [ ]:
# ---------------------------
# Alternative Hybrid Training (LS+ gradient descent on all)
# ---------------------------
def train_hybrid(model, X, y, epochs=100, lr=1e-4):
    # Step 1: LS for consequents
    train_ls(model, X, y)
    # Step 2: GD fine-tuning
    train_gd(model, X, y, epochs=epochs, lr=lr)

In [ ]:
GD_model = TSK(n_inputs=Xtr.shape[1], n_rules=n_clusters, centers=centers[:,:-1], sigmas=sigmas[:,:-1])
ANFIS_model = TSK(n_inputs=Xtr.shape[1], n_rules=n_clusters, centers=centers[:,:-1], sigmas=sigmas[:,:-1])
Hybrid_model = TSK(n_inputs=Xtr.shape[1], n_rules=n_clusters, centers=centers[:,:-1], sigmas=sigmas[:,:-1])

In [ ]:
#Training with GD
train_gd(GD_model, Xtr, ytr.reshape(-1,1), epochs=100, lr=1e-3)

In [ ]:
#Training with ANFIS
train_hybrid_anfis(ANFIS_model, Xtr, ytr.reshape(-1,1), max_iters=10, gd_epochs=20, lr=1e-3)

In [ ]:
#Training Hybrid Classic 
train_hybrid(Hybrid_model, Xtr, ytr.reshape(-1,1), epochs=100, lr=1e-4)

In [1]:
#Predictions
y_pred_GD, _, _ = GD_model(Xte)
y_pred_ANFIS, _, _ = ANFIS_model(Xte)
y_pred_Hybrid, _, _ = Hybrid_model(Xte)

# Compute MSE for each
mse_GD = mean_squared_error(yte.detach().numpy(), y_pred_GD.detach().numpy())
mse_ANFIS = mean_squared_error(yte.detach().numpy(), y_pred_ANFIS.detach().numpy())
mse_Hybrid = mean_squared_error(yte.detach().numpy(), y_pred_Hybrid.detach().numpy())

print(f"MSE GD: {mse_GD}")
print(f"MSE ANFIS: {mse_ANFIS}")
print(f"MSE Hybrid: {mse_Hybrid}")

### Classification

In [ ]:
# Load dataset
diabetes = datasets.fetch_openml("diabetes", version=1, as_frame=True)

# Get the full DataFrame (features + target)
df = diabetes.frame

# Replace string target with binary values directly
df["class"] = df["class"].map({
    "tested_negative": 0,
    "tested_positive": 1
})

# Define X and y from the updated DataFrame
X = df.drop(columns="class").values
y = df["class"].values

print(y[:10])
df.head()

In [ ]:
#train test spliting
test_size=0.2
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
# Standardize features
scaler=StandardScaler()
Xtr= scaler.fit_transform(Xtr)
Xte= scaler.transform(Xte)

In [ ]:
# Number of clusters 
n_clusters = 2
m= 1.5

# Concatenate target for clustering
Xexp=np.concatenate([Xtr, ytr.reshape(-1, 1)], axis=1)
#Xexp=Xtr

# Transpose data for skfuzzy (expects features x samples)
Xexp_T = Xexp.T 

# Fuzzy C-means clustering
centers, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    Xexp_T, n_clusters, m=m, error=0.005, maxiter=1000, init=None,
)

In [ ]:
# Compute sigma (spread) for each cluster
sigmas = []
for j in range(n_clusters):
    # membership weights for cluster j, raised to m
    u_j = u[j, :] ** m
    # weighted variance for each feature
    var_j = np.average((Xexp - centers[j])**2, axis=0, weights=u_j)
    sigma_j = np.sqrt(var_j)
    sigmas.append(sigma_j)
sigmas=np.array(sigmas)# Hard clustering from fuzzy membership

In [ ]:
# Hard clustering from fuzzy membership
cluster_labels = np.argmax(u, axis=0)
print("Fuzzy partition coefficient (FPC):", fpc)

cluster_labels = np.argmax(u, axis=0)  

# Turn Xexp into a dataframe for plotting
df = pd.DataFrame(Xexp, columns=[f"Feature {i}" for i in range(Xexp.shape[1])])
df["Cluster"] = cluster_labels

# Pairplot with hue = cluster
sns.pairplot(df, vars=df.columns[:-1], hue="Cluster", palette="tab10", diag_kind="kde")
plt.show()

In [ ]:
GD_model = TSK(n_inputs=Xtr.shape[1], n_rules=n_clusters, centers=centers[:,:-1], sigmas=sigmas[:,:-1])
ANFIS_model = TSK(n_inputs=Xtr.shape[1], n_rules=n_clusters, centers=centers[:,:-1], sigmas=sigmas[:,:-1])
Hybrid_model = TSK(n_inputs=Xtr.shape[1], n_rules=n_clusters, centers=centers[:,:-1], sigmas=sigmas[:,:-1])

In [ ]:
#Training with GD
train_gd(GD_model, Xtr, ytr.reshape(-1,1), epochs=100, lr=1e-3)

In [ ]:
#Training with ANFIS
train_hybrid_anfis(ANFIS_model, Xtr, ytr.reshape(-1,1), max_iters=10, gd_epochs=20, lr=1e-3)

In [ ]:
#Training Hybrid Classic 
train_hybrid(Hybrid_model, Xtr, ytr.reshape(-1,1), epochs=100, lr=1e-4)

In [2]:
# Predictions
y_pred_GD = (GD_model(Xte)[0] >= 0.5).int()
y_pred_ANFIS = (ANFIS_model(Xte)[0] >= 0.5).int()
y_pred_Hybrid = (Hybrid_model(Xte)[0] >= 0.5).int

# Compute Accuracy for each
acc_GD = accuracy_score(yte.detach().numpy(), y_pred_GD.detach().numpy())
acc_ANFIS = accuracy_score(yte.detach().numpy(), y_pred_ANFIS.detach().numpy())
acc_Hybrid = accuracy_score(yte.detach().numpy(), y_pred_Hybrid.detach().numpy())

print(f"Accuracy GD: {acc_GD:.4f}")
print(f"Accuracy ANFIS: {acc_ANFIS:.4f}")
print(f"Accuracy Hybrid: {acc_Hybrid:.4f}")

NameError: name 'GD_model' is not defined